In [40]:
import gzip
from sklearn.neighbors import KNeighborsClassifier
import pickle
import multiprocessing

In [48]:
N_SAMPLES = 500
NUM_PROCESSES = 10

with open(f"data/sentiment-dataset-{N_SAMPLES}.pickle", "rb") as f:
    dataset = pickle.load(f)

train_x, train_y, test_x, test_y = dataset

In [49]:
len(train_x)

401

In [50]:
print(train_x[1], "\n", train_y[1])

Seriously disappointing performance by Brad Pitt and Q T, the plot is very superficial and lame, and, unless indirectly intended, this film actually glorify the Nazis and portrays them as men of honor, and show that the Jewish people are deceiving, cant keep promises and bloody vicious. ((THE FOLLOWING CONTAINS SPOILER)) Hitler together with the most notorious Nazis are attending a stupid plot less movie about the killing of 300 Italian soldiers in a small cinema theater in Paris is unbelievably ridiculous. the Nazis laughing and hooraying each killing in the movie as if watching a basketball game STUPID, the deal at the end is lame. whats really appalling is that the movie earned great reviews and is ranked here in the 40s amongst the greatest 250 films. will not be surprised if it harvested many awards, including Oscars, as well. the movie is simply a kissing ass to the Jewish people, but hey reconsider, its not even doing a great job doing that. it truly dwarfed the whole Nazi - Jew

In [51]:
x_compresed = len(gzip.compress(train_x[0].encode()))
x_compresed

451

In [52]:
len(train_x[0])

742

In [53]:
x2_compresed = len(gzip.compress(train_x[1].encode()))
x2_compresed

639

In [54]:
xx2 = len(gzip.compress((" ".join([train_x[0], train_x[1]]).encode())))
xx2

1024

### Normalized Compression Distance

In [55]:
ncd = (xx2 - min(x_compresed, x2_compresed)) / max(x_compresed, x2_compresed)
ncd

0.8967136150234741

doing it for all the samples

In [64]:
def ncd(x1, x2):
    x1_compressed = len(gzip.compress(x1.encode()))
    x2_compressed = len(gzip.compress(x2.encode()))
    x1x2 = len(gzip.compress((" ".join([x1, x2]).encode())))
    return (x1x2 - min(x1_compressed, x2_compressed)) / max(x1_compressed, x2_compressed)

In [65]:
train_ncd = [[ncd(train_x[i], train_x[j]) for j in range(len(train_x))] for i in range(len(train_x))]

In [66]:
test_ncd = [[ncd(test_x[i], train_x[j]) for j in range(len(train_x))] for i in range(len(test_x))]

In [67]:
len(train_ncd), len(test_ncd)

(401, 101)

In [68]:
knn_classifier = KNeighborsClassifier(n_neighbors=7)
knn_classifier.fit(train_ncd, train_y)

KNeighborsClassifier(n_neighbors=7)

In [69]:
print("Accuracy: ", knn_classifier.score(test_ncd, test_y))

Accuracy:  0.7029702970297029


In [70]:
# Initialize NCD matrices
train_ncd = [[0] * len(train_x) for _ in range(len(train_x))]
test_ncd = [[0] * len(train_x) for _ in range(len(test_x))]

In [73]:
# Function to calculate NCD row
# input is a tuple of (index, text)
# returns index and row's ncd vector
def calc_ncd_row(data_row):
    i = data_row[0]
    text = data_row[1]
    row = [ncd(text, train_x[j]) for j in range(len(train_x))]
    return i, row 

In [74]:
with multiprocessing.Pool(NUM_PROCESSES) as pool:
    # compute train NCD
    train_data = enumerate(train_x)
    train_results = pool.map(calc_ncd_row, train_data)

    # compute test NCD
    test_data = enumerate(test_x)
    test_results = pool.map(calc_ncd_row, test_data)

Process SpawnPoolWorker-50:
Process SpawnPoolWorker-51:
Process SpawnPoolWorker-48:
Process SpawnPoolWorker-52:
Process SpawnPoolWorker-49:
Process SpawnPoolWorker-53:
Traceback (most recent call last):
  File "/Users/abhishekanand/pytorch-env/env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/abhishekanand/pytorch-env/env/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/abhishekanand/pytorch-env/env/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/abhishekanand/pytorch-env/env/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'calc_ncd_row' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Users/abhishekanand/pytorch-env/env/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "

KeyboardInterrupt: 

In [ ]:
for i, row in train_results:
    train_ncd[i] = row
    for i, row in test_results:
        test_ncd[i] = row